# NPI to Buckets
This script pulls data from the *NPPES* (National Plan and Provider Enumeration System). Given a physician's NPI, it categorizes his/her specialty as one of the following: *anesthesiology*, *radiology*, *general practice*, or none of the previous buckets.

In [23]:
import pandas as pd
import requests
from sys import exit

The following table displays each specialties' underlying taxonomy codes, used to categorize the physician.

In [29]:
taxonomy_codes = pd.read_excel("https://www.cms.gov/OpenPayments/Downloads/Physician-Taxonomy-Crosswalk.xlsx", header = 3)
taxonomy_codes

['PROVIDER TAXONOMY CODE',
 'PROVIDER TYPE DESCRIPTION',
 'CLASSIFICATION ',
 'SPECIALIZATION']

In [26]:
num = input("Enter the physician's NPI: ")
# Returns JSON array with information about physician
resp = requests.get("https://npiregistry.cms.hhs.gov/api/?number="+num+"&enumeration_type=&taxonomy_description=&first_name=&last_name=&organization_name=&address_purpose=&city=&state=&postal_code=&country_code=&limit=&skip")

try:
    resp.json()['results']
except:
    print("Invalid NPI")
    exit()
else:
    print("Loading...")

data = pd.io.json.json_normalize(resp.json()['results'], 'taxonomies')

# Taxonomy Codes mapped to specific specialties
anesth = taxonomy_codes[taxonomy_codes["CLASSIFICATION "] == "Anesthesiology"]["PROVIDER TAXONOMY CODE"].tolist()
radiol = taxonomy_codes[taxonomy_codes["CLASSIFICATION "] == "Radiology"]["PROVIDER TAXONOMY CODE"].tolist()
genprac = taxonomy_codes[taxonomy_codes["CLASSIFICATION "] == "General Practice"]["PROVIDER TAXONOMY CODE"].tolist()

code = data['code'].to_string(index = False)
if code in anesth:
    print("Anesthesiologist")
elif code in radiol:
    print("Radiologist")
elif code in genprac:
    print("General practitioner")
else:
    print("Not bucketed")

Enter the physician's NPI: 1538176961
Loading...
Anesthesiologist
